 Название | Год | Кассовый сбор в прокате | Лучший фильм | Лучший режиссер | Лучшая операторская работа | Лучшие визуальные эффекты | Лучший звук | Лучшая работа художника-постановщика | Лучший дизайн костюмов | Другие


In [1]:
import requests
import json
from bs4 import BeautifulSoup
from kinopoisk_unofficial.kinopoisk_api_client import KinopoiskApiClient
from kinopoisk_unofficial.request.films.film_request import FilmRequest
from kinopoisk_unofficial.request.films.box_office_request import BoxOfficeRequest
import time
from selenium import webdriver
import pandas as pd

In [5]:
browser = webdriver.Chrome('/Users/kosarenok/Desktop/chromedriver_mac64/chromedriver')

url_api = "https://kinopoiskapiunofficial.tech/api/v2.2/films/"
url = "https://www.kinopoisk.ru/lists/movies/box-world-not-usa/?page=2"

browser.get(url)

# r = requests.get(url)

soup = BeautifulSoup(browser.page_source, 'lxml')

/var/folders/n0/z4q56_291yl18hx7bzl2b5b40000gn/T/ipykernel_36990/3263415282.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('/Users/kosarenok/Desktop/chromedriver_mac64/chromedriver')


In [3]:
links = []
films_raw_data = []

In [6]:
for a in soup.findAll('a', class_='base-movie-main-info_link__YwtP1'):
    links.append(a['href'].split('/')[2])

In [13]:
api_client = KinopoiskApiClient('9eaa3824-1c1b-41ee-850e-e6b20e2ed3fc')

counter = 0

for i in range(0,len(links)):
    counter += 1
    if counter <= 10:
        request = FilmRequest(int(links[i]))
        response = api_client.films.send_film_request(request)
        films_raw_data.append(response)
    else:
        time.sleep(2)
        counter = 0

for i in range(0, len(films_raw_data)):
    films_raw_data[i] = films_raw_data[i].film

In [14]:
films_budget_data = []

counter = 0

for j in range(0,len(films_raw_data)):
    counter += 1
    if counter <= 10:
        request = BoxOfficeRequest(films_raw_data[j].kinopoisk_id)
        response = api_client.films.send_box_office_request(request)
        film_budget = {
            'kino_id' : films_raw_data[j].kinopoisk_id,
            'type' : response.items[[x for x in range(len(response.items)) if response.items[x].type == 'WORLD'][0]].type,
            'amount' : response.items[[x for x in range(len(response.items)) if response.items[x].type == 'WORLD'][0]].amount,
            'Currency' : response.items[[x for x in range(len(response.items)) if response.items[x].type == 'WORLD'][0]].currency_code
        }
        films_budget_data.append(film_budget)
    else:
        time.sleep(2)
        counter = 0

In [15]:
films_award_data = []

headers = {'X-API-KEY': '9eaa3824-1c1b-41ee-850e-e6b20e2ed3fc'}
counter = 0

for k in range(0,len(films_raw_data)):
    counter += 1
    if counter <= 10:
        request = requests.get(f'https://kinopoiskapiunofficial.tech/api/v2.2/films/{films_raw_data[k].kinopoisk_id}/awards', headers=headers)
        response = request.json()
        film_awards = {
                'kino_id' : films_raw_data[k].kinopoisk_id,
                'name' : 'Британская академия'
            }
        for t in [x for x in range(len(response['items'])) if response['items'][x]['name'] == 'Британская академия']:
            film_awards[response['items'][t]['nominationName']] = response['items'][t]['win']
            film_awards['year'] = response['items'][t]['year']
            films_award_data.append(film_awards)
    else:
        time.sleep(2)
        counter = 0

In [16]:
films_award_dataframe = pd.DataFrame(films_award_data)
films_award_dataframe = films_award_dataframe.drop_duplicates()
films_dataframe = pd.DataFrame(films_raw_data)
films_budget_dataframe = pd.DataFrame(films_budget_data)

In [17]:
films_budget_dataframe.rename(columns = {'kino_id':'kinopoisk_id'}, inplace=True)

films_award_dataframe.rename(columns = {'kino_id':'kinopoisk_id'}, inplace=True)

In [18]:
films_with_budget = films_dataframe.merge(films_budget_dataframe, how='outer', on='kinopoisk_id')

In [19]:
all_films_data = films_with_budget.merge(films_award_dataframe, how='outer', on='kinopoisk_id')

In [20]:
all_films_data.to_excel("Kinopoisk_bafta_parsed_data_new.xlsx", sheet_name='raw_data')  